<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">The EM algorithm and its applications</div>

In statistics, an expectation–maximization (EM) algorithm is an iterative method to find maximum likelihood  estimates of parameters in statistical models, where the model depends on unobserved latent variables. The EM iteration alternates between performing an expectation (E) step, which creates a function for the expectation of the log-likelihood evaluated using the current estimate for the parameters, and a maximization (M) step, which computes parameters maximizing the expected log-likelihood found on the E step. These parameter-estimates are then used to determine the distribution of the latent variables in the next E step.

Through this notebook, we will discover this algorithm, from its mathematial basis to its application to a clustering problem.



1. [An unsupervised problem : the Old Faithful geyser](#sec1)
2. [How the EM algorithm works](#sec2)
3. [Gaussian Processes for classification](#sec3)


# 1. <a id="sec1"></a>An unsupervised problem : the Old Faithful geyser

Located in the Yellowstone National Park, the Old Faithful geyser holds its name from the high predictibility of its two main features : the duration of its eruptions, and the time building up to them. One dataset of 273 eruptions can be represented with these two parameters :

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
fig_size=(10, 10)

res = np.loadtxt("OldFaithful.csv", delimiter=',', skiprows = 1)
X = res[:,0:-1]
y = res[:,-1].astype(int)

fig=plt.figure(figsize=fig_size, dpi= 80, facecolor='w', edgecolor='k')
plt.xlabel('Eruption Duration (min)')
plt.ylabel('Time before Eruption (min)')
plt.scatter(X,y,c='b',s=20)

We observe that the geyser is bimodal, which means every eruption can be caracterised as being of one form or another, that we will denominate as "long" or "short". At seeing the previous plot, we are itching to try to draw a frontier between the two modes. We are going to try to do even better : we want to classify every eruption in one category, but we also will determine the parameters of the probability laws ruling each eruption regime. 

We will now study a type of algorithm particularly fit for this **unsupervised** problem.


# <a id="sec2"></a>2. How the EM algorithm works

Let $\mathbf{X}$ be a random vector which results from a parameterized family. We wish to find the parameters $\theta$ such that $\mathbb{P}(\mathbf{X}|\theta)$ is a maximum. This is known as the Maximum Likelihood (ML) estimate for $\theta$. In order to estimate $\theta$, we introduce as we frequenlty do the log likelihood function defined as : 

$$L(\theta) = \ln (\mathbb{P}(\mathbf{X}|\theta))$$


The likelihood function is considered to be a function of the parameter $\theta$ given the data $\mathbf{X}$.

### Idea of the algorithm
The EM algorithm is an efficient iterative procedure to compute the Maximum Likelihood (ML) estimate in the presence of missing or hidden data. In ML estimation, we wish to **estimate the model parameters for which the observed data are the most likely**. Each iteration of the EM algorithm consists of two processes: the E-step  creates a function for the expectation of the log-likelihood evaluated using the current estimate for the parameters, and the M-step then maximizes this function. 

Let $\theta_n$ be the estimated parameters at the end of the $n^{th}$ iteration. We will detail how the algorithm proceeds to find the next value $\theta_{n+1}$. 

### E-step
In the expectation, or E-step, the missing data are estimated given the observed data and current estimate of the model parameters. This is achieved using the conditional expectation, explaining the choice of terminology.

We want to choose $\theta$ such that we have an increase in likelihood as big as possible, i.e. we want $L(\theta)$ to be greater than $L(\theta_n)$. Thus we wish to maximize the following quantity :

$$L(\theta) − L(\theta_n) = \ln(\mathbb{P}(\mathbf{X}|\theta)) − \ln(\mathbb{P}(\mathbf{X}|\theta_n))$$

So far, we have not considered any unobserved or missing variables. In problems where such data exist, the EM algorithm provides a natural framework for their inclusion. Alternately, hidden variables may be introduced purely as an artifice for making the maximum likelihood estimation of θ easier to compute. Frequently, in unsupervised cases, the missing value will be the class of each point. In this case, it is assumed that knowledge of the hidden variables will make the maximization of the likelihood function easier. Either way, denote the hidden random vector by $\mathbf{Z}$ and a given realization by $\mathbf{z}$. We can now write the previous quantity to maximize in the following way :

$$L(\theta) − L(\theta_n) = \ln  \sum\limits_{z} \mathbb{P}(\mathbf{X}|\mathbf{z},\theta)\mathbb{P}(\mathbf{z}|\theta) − \ln(\mathbb{P}(\mathbf{X}|\theta_n))$$

We can now manipulate the equation in the following way, in order to be able to apply Jensen's inequality later on :

$$L(\theta) − L(\theta_n) = \ln  \sum\limits_{z} \mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \Big(\frac{\mathbb{P}(\mathbf{X}|\mathbf{z},\theta) \mathbb{P}(\mathbf{z}|\theta)}{\mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n)}\Big) − \ln(\mathbb{P}(\mathbf{X}|\theta_n))$$

The coefficients of the left-hand term are now such that we can apply Jensen's inequality (that uses the concavity of the logarithm) :  

$$L(\theta) − L(\theta_n) \geq \sum\limits_{z} \mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \ln \Big( \frac{\mathbb{P}(\mathbf{X}|\mathbf{z},\theta) \mathbb{P}(\mathbf{z}|\theta)}{\mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n)} \Big) − \ln(\mathbb{P}(\mathbf{X}|\theta_n))$$

This can finally be rewritten : 

$$L(\theta) \geq L(\theta_n) + \sum\limits_{z} \mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \ln \Big( \frac{\mathbb{P}(\mathbf{X}|\mathbf{z},\theta) \mathbb{P}(\mathbf{z}|\theta)}{\mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \mathbb{P}(\mathbf{X}|\theta_n)} \Big) = l(\theta|\theta_n)$$

Where we introduced $l(\theta|\theta_n)$ for convenience.

We can now notice two important details : firstly, the function $l(\theta|\theta_n)$ is upper-bounded by the likelihood $L(\theta)$, and secondly they are equal when evaluated at $\theta = \theta_n $. Therefore, **any $\theta$ which increases $l(\theta|\theta_n)$ also increases $L(\theta)$**. In order to achieve the greatest possible increase in the value of $L(\theta)$, the EM algorithm calls for selecting $\theta$ such that $l(\theta|\theta_n)$ is maximized. This is done in the M-step.

### M-step
In the M-step, the likelihood function is maximized under the assumption that the missing data are known. The estimate of the missing data from the E-step are used in lieu of the actual missing data. Convergence is assured since the algorithm is guaranteed to increase the likelihood at each iteration.

We will choose $\theta_{n+1}$ such that : 

$$\theta_{n+1} = \arg \max_{\theta} l(\theta|\theta_n) = \arg \max_{\theta} \bigg(L(\theta_n) + \sum\limits_{z} \mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \ln \Big( \frac{\mathbb{P}(\mathbf{X}|\mathbf{z},\theta) \mathbb{P}(\mathbf{z}|\theta)}{\mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \mathbb{P}(\mathbf{X}|\theta_n)} \Big) \bigg)$$

Now, dropping terms that are constant with respect to $\theta$ gives :

\begin{equation}
  \begin{aligned}
    \theta_{n+1} & = \arg \max_{\theta} \bigg(\sum\limits_{z} \mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \ln \big( \mathbb{P}(\mathbf{X}|\mathbf{z},\theta) \mathbb{P}(\mathbf{z}|\theta) \big) \bigg) \\
      & = \arg \max_{\theta} \bigg(\sum\limits_{z} \mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \ln \big( \frac{\mathbb{P}(\mathbf{X},\mathbf{z},\theta)}{\mathbb{P}(\mathbf{z},\theta)} \frac{\mathbb{P}(\mathbf{z},\theta)}{\mathbb{P}(\theta)} \big) \bigg) \\
      & = \arg \max_{\theta} \bigg(\sum\limits_{z} \mathbb{P}(\mathbf{z}|\mathbf{X},\theta_n) \ln \mathbb{P}(\mathbf{X},\mathbf{z}|\theta)  \bigg) \\
      & = \arg \max_{\theta} \bigg(\mathbb{E}_{\mathbf{Z}|\mathbf{X}, \theta} \Big[ \ln \mathbb{P}(\mathbf{X},\mathbf{z}|\theta) \Big]  \bigg) 
  \end{aligned}
\end{equation}

Thus, $\mathbb{E}_{\mathbf{Z}|\mathbf{X}, \theta} \Big[ \ln \mathbb{P}(\mathbf{X},\mathbf{z}|\theta) \Big] $ is the quantity we are looking to maximize in the M-step, that was derived in the E-step.

How do we maximize this expression ? One approach is to use Lagrange multipliers, and another, arguably more elegant approach is to use techniques linked to Kullback-Leibler divergence. The maximization in itself is out of the scope of this notebook, but further detail is given in chapter 20 of the book __Bayesian Reasoning and Machine Learning__ by David Barber (link : http://web4.cs.ucl.ac.uk/staff/D.Barber/textbook/091117.pdf ). 


### Characteristics of the EM algorithm

The EM algorithm, contrary to other algorithms aiming to derive the Maximum of Likelihood, **does not require to derivate the likelihood**. This makes it a very strong candidate when this differenciation is complex.

It will converge to a **local** maximum of the likelihood. This means that either some metaheuristic such as simulated annealing should be used, or more simply that it should be ran several times with different initial values of $\theta$ in order to find the general maximum of likelihood.

The general idea of the EM algorithm is implemented in many functions, depending on what form of distributions are expected. In particular, we will study in the following section **Gaussian Mixtures Models**, where the distributions are assumed to be gaussians and we are looking for their parameters.

EM is frequently used for data clustering in machine learning and computer vision. In natural language processing, two prominent instances of the algorithm are the Baum-Welch algorithm for hidden Markov models, and the inside-outside algorithm for unsupervised induction of probabilistic context-free grammars.


# <a id="sec3"></a>3. Gaussian Mixtures Models for eruption classification

Gaussian Mixtures Models are a particular type of EM algorithms. In this situation, two gaussians are mixed, and we are trying to find their means and variances  through the observed values. Thus this case is perfetly adapted to our geyser issue. Luckily, a Gaussian Mixture solver is inluded in Scikit-Learn ( https://scikit-learn.org/stable/modules/mixture.html ).

First let's look back at the data :

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
fig_size=(10, 10)

res = np.loadtxt("OldFaithful.csv", delimiter=',', skiprows = 1)
X = res[:,0:-1]
y = res[:,-1].astype(int)

fig=plt.figure(figsize=fig_size, dpi= 80, facecolor='w', edgecolor='k')
plt.xlabel('Eruption Duration (min)')
plt.ylabel('Time before Eruption (min)')
plt.scatter(X,y,c='b',s=20)

Now, we can try to fit a GaussianMixture with this set. In the specific case of a gaussian mixture, the steps of the EM algorithm are clearer. First one assumes random components (randomly centered on data points, learned from k-means, or even just normally distributed around the origin) and computes for each point a probability of being generated by each component of the model (E-step). Then, one tweaks the parameters to maximize the likelihood of the data given those assignments (M-step). Thus, the "hidden" data $\mathbf{Z}$ introduced in the precedent section is here the type of eruption. 

Let's  instanciate a GaussianMixture :

In [ ]:
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal
GMM = GaussianMixture(n_components=2,  tol = 1e-6, max_iter = 50, init_params = 'random',random_state = 1, verbose = 1) # Instantiate and fit the model
GMM.get_params

This GaussianMixture was set to look for two gaussians, on maximum 50 iterations, and with a convergence threshold of 1e-6. The initialisation is random, but the seed is not, in order to compare several runs with different parameters. Change the seed if you want to see convergence for other initial values. 

<div class="alert alert-warning">**Note:**<br>
In this example, the initialization was chosen to be **random** in order to see what the algorithm does. 
Indeed, since our toy example is quite simple, an initialization with k-means would do almost all the job
and wouldn't get to show the real impact of the EM optimization.
</div>

Now we will fit the GaussianProcess and look at the results :

In [ ]:
GMM.fit(res)
means = GMM.means_ 
covariances = GMM.covariances_

print("\nMoyennes : " + str(means))
print("\nMatrices de covariance : " + str(covariances))

# Plot   
x,y = np.meshgrid(np.sort(res[:,0]),np.sort(res[:,1]))
XY = np.array([x.flatten(),y.flatten()]).T
fig = plt.figure(figsize=(10,10))
ax0 = fig.add_subplot(111)
ax0.scatter(res[:,0],res[:,1])
for m,c in zip(means,covariances):
    multi_normal = multivariate_normal(mean=m,cov=c)
    ax0.contour(np.sort(res[:,0]),np.sort(res[:,1]),multi_normal.pdf(XY).reshape(len(X),len(X)),colors='black',alpha=0.3)
    ax0.scatter(m[0],m[1],c='grey',zorder=10,s=100)
    
plt.xlabel('Eruption Duration (min)')
plt.ylabel('Time before Eruption (min)')
plt.show()

As we can see, the EM algorithm converged, and the two modes of eruption are represented by two gaussians.

Now we can give the probability that a given eruption is of the first group or the second.

In [ ]:
#Eruption 1, orange dot
Y1 = np.array([[4.5],[80]])
prediction = GMM.predict_proba(Y1.T)
print("Prediction 1 (orange dot) : " + str(prediction))

#Eruption 2, red dot
Y2 = np.array([[3],[60]])
prediction = GMM.predict_proba(Y2.T)
print("Prediction 2 (red dot) : " + str(prediction))

# Plot   
x,y = np.meshgrid(np.sort(res[:,0]),np.sort(res[:,1]))
XY = np.array([x.flatten(),y.flatten()]).T
fig = plt.figure(figsize=(10,10))
ax0 = fig.add_subplot(111)
ax0.scatter(res[:,0],res[:,1])
ax0.scatter(Y1[0,:],Y1[1,:],c='orange',zorder=10,s=100)
ax0.scatter(Y2[0,:],Y2[1,:],c='red',zorder=10,s=100)
for m,c in zip(means,covariances):
    multi_normal = multivariate_normal(mean=m,cov=c)
    ax0.contour(np.sort(res[:,0]),np.sort(res[:,1]),multi_normal.pdf(XY).reshape(len(X),len(X)),colors='black',alpha=0.3)
    ax0.scatter(m[0],m[1],c='grey',zorder=10,s=100)

plt.xlabel('Eruption Duration (min)')
plt.ylabel('Time before Eruption (min)')    
plt.show()

We thus see that, for example, an eruption building up for 60 min and lasting 3 min as a 66% chance of having been produced by the short mechanism.

<div class="alert alert-warning">**Exercice:**<br>
Understanding this algorithm really demands hands-on training, so now try to play with the GaussianMixture ! 
Using the same dataset, try to change every parameter when instanciating the regressor, and look at the impact. <br>
In particular, look at the **impact of the maximum number of iterations**, and find where is the limit of convergence. Progressively adding iterations will also show you how each EM iteration tweaks the parameters of the gaussians.
</div>

## Sources

Maximum Likelihood from Incomplete Data via the EM Algorithm, Dempster et. al., 1977 <br>
The Expectation Maximization Algorithm A short tutorial, S. Borman, 2004 <br>
Bayesian Reasoning and Machine Learning, D. Barber, 2017 <br>
Wikipedia page (https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) <br>


(these papers can be found in the notebook folder)